# **Install YOLOv8:**

In [ ]:
!pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install tensorflow
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.4 MB/s eta 0:00:00


In [ ]:
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

In [ ]:
def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images


    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')



    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")



    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
train_test_split('/content/drive/MyDrive/yolo8/data/') ### without negative images
# train_test_split('./data/','./negative_images/') ### if you want to feed negative images


------ PROCESS STARTED -------
--- This folder has a total number of 692 images---


  0%|          | 0/554 [00:00<?, ?it/s]

------ Training data created with 80% split 554 images -------


  0%|          | 0/138 [00:00<?, ?it/s]

------ Testing data created with a total of 138 images ----------
------ TASK COMPLETED -------


In [ ]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.112 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 23.7/78.2 GB disk)


In [ ]:
!yolo detect train data=/content/drive/MyDrive/yolo8/dataset.yaml model=yolov8n.pt epochs=10 imgsz=640 batch=8 project=/content/drive/MyDrive/yolo8/training_results name=vehicle

Ultralytics YOLOv8.0.112 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/yolo8/dataset.yaml, epochs=10, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolo8/training_results, name=vehicle, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=tor

In [ ]:
!yolo detect predict model=/content/drive/MyDrive/yolo8/training_results/vehicle/weights/best.pt conf=0.55 source=/content/drive/MyDrive/yolo8/test_images

Ultralytics YOLOv8.0.112 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006623 parameters, 0 gradients

image 1/8 /content/drive/MyDrive/yolo8/test_images/img1027.jpg: 640x640 1 Car, 7.4ms
image 2/8 /content/drive/MyDrive/yolo8/test_images/img1028.jpg: 640x640 1 Car, 7.4ms
image 3/8 /content/drive/MyDrive/yolo8/test_images/img1030.jpg: 640x640 2 Cars, 7.4ms
image 4/8 /content/drive/MyDrive/yolo8/test_images/img1037.jpg: 640x640 2 Cars, 7.4ms
image 5/8 /content/drive/MyDrive/yolo8/test_images/img1038.jpg: 640x640 2 Cars, 7.4ms
image 6/8 /content/drive/MyDrive/yolo8/test_images/img1040.jpg: 640x640 1 Bicycle, 7.3ms
image 7/8 /content/drive/MyDrive/yolo8/test_images/img1047.jpg: 640x640 (no detections), 7.4ms
image 8/8 /content/drive/MyDrive/yolo8/test_images/img1048.jpg: 640x640 3 Cars, 7.5ms
Speed: 2.9ms preprocess, 7.4ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict


In [ ]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/yolo8/output

In [ ]:
!yolo detect predict model=/content/drive/MyDrive/yolo8/training_results/vehicle/weights/best.pt conf=0.77 source=/content/drive/MyDrive/yolo8/test_videos

Streaming output truncated to the last 5000 lines.
video 2/2 (46204/51201) /content/drive/MyDrive/yolo8/test_videos/m6motorwaytraffic.mp4: 384x640 2 Cars, 7.0ms
video 2/2 (46205/51201) /content/drive/MyDrive/yolo8/test_videos/m6motorwaytraffic.mp4: 384x640 1 Car, 6.8ms
video 2/2 (46206/51201) /content/drive/MyDrive/yolo8/test_videos/m6motorwaytraffic.mp4: 384x640 1 Car, 6.8ms
video 2/2 (46207/51201) /content/drive/MyDrive/yolo8/test_videos/m6motorwaytraffic.mp4: 384x640 1 Car, 7.1ms
video 2/2 (46208/51201) /content/drive/MyDrive/yolo8/test_videos/m6motorwaytraffic.mp4: 384x640 1 Car, 7.1ms
video 2/2 (46209/51201) /content/drive/MyDrive/yolo8/test_videos/m6motorwaytraffic.mp4: 384x640 2 Cars, 7.4ms
video 2/2 (46210/51201) /content/drive/MyDrive/yolo8/test_videos/m6motorwaytraffic.mp4: 384x640 1 Car, 7.3ms
video 2/2 (46211/51201) /content/drive/MyDrive/yolo8/test_videos/m6motorwaytraffic.mp4: 384x640 1 Car, 7.3ms
video 2/2 (46212/51201) /content/drive/MyDrive/yolo8/test_videos/m6motorway

In [ ]:
!cp -r /content/runs/detect/predict3 /content/drive/MyDrive/yolo8/output

In [ ]:
tf_model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", from_pt=True)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tf_model.save_pretrained("bert-base-uncased")

In [ ]:
!cp -r /content/bert-base-uncased/tf_model.h5 /content/drive/MyDrive/yolo8